### 고객의 재구매율 분석
> 전체 구매자와 재구매 이력이 있는 구매자, 그중 웰컴쿠폰을 쓴 구매자의 비교
- 전체 구매자와 재구매 이력이 있는 구매자의 비율은 월별로 비슷했다.
- 웰컴 쿠폰을 사용한 경우 4, 5, 6월에 집중되있는것을 확인
- 프로그래밍 카테고리의 비중이 가장 높았음


- 구매 테스트를 위한 페이크강의가 존재하는것 같다.

>To_do
- 전체 데이터에 대한 전처리 필요
- 웰컴쿠폰과 다른 쿠폰과의 구매 비교
- 분야별 구매자들의 재구매이력과 쿠폰 사용률 분석


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


In [ ]:
plt.rc('font', family='NanumBarunGothic') 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_c= pd.read_excel("/content/drive/MyDrive/colab notebooks/파이널프로젝트_RAW_210329_210926.xlsx")


In [ ]:
df = df_c

## 데이터전처리
- type 칼럼의 ORDER, TRANSACTION 값을 가진 ROW drop

In [ ]:
d = df[df.type.str.contains('ORDER|TRANSCATION')].index
df.drop(d, inplace = True)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

## 강의를 결제완료한 사람들의 Df 생성

In [ ]:
pay = df[(df.type == 'PAYMENT') & (df.state == 'COMPLETED')]

In [ ]:
pay

In [ ]:
pay['date'] = pay['completed_at'].dt.month

In [ ]:
pay.customer_id.value_counts()

#### 39519명이 강의를 결제함

### 결제완료한 사람 중 2번이상 결제한 사람을 대상으로 df생성

In [ ]:
cond2 = pay.customer_id.value_counts().loc[pay.customer_id.value_counts() > 1].index
cond2

In [ ]:
pay.customer_id.value_counts().loc[pay.customer_id.value_counts() > 1].count()/pay.customer_id.value_counts().count()

- 전체 유저 중 2번이상 이용한 유저의 비율 약 20%

In [ ]:
df1 = pay.loc[pay.customer_id.isin(cond2)]

### 2번이상 결제한 사람들의 강의중 쿠폰을 사용한 강의를 대상으로 df생성

In [ ]:
coupon_y = df1[df1.coupon_title.notnull()].customer_id
coupon_y

In [ ]:
df2 = df1.loc[df1.customer_id.isin(coupon_y)]

In [ ]:
df2

In [ ]:
df3 = df2.dropna(axis=0)

In [ ]:
df3

In [ ]:
df3.groupby('category_title').size()

In [ ]:
df3.groupby('coupon_title').size().sort_values(ascending = False)

In [ ]:
cond3 = df3[df3.coupon_title.str.contains('웰컴쿠폰')].customer_id

In [ ]:
df4 = pay.loc[pay.customer_id.isin(cond3)]

- 2번 이상의 구매를 하고 웰컴쿠폰을 쓴 이력이 있는 구매자들

In [ ]:
df4

### 재수강율?
재수강한 고객 / 전체 고객

쿠폰?

분야?

In [ ]:
f,ax = plt.subplots(1,2,figsize=(16,8))
pay['date'].value_counts().plot.pie(autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Month Percentage')
ax[0].set_ylabel('')
sns.countplot('date', data=pay, ax=ax[1])
ax[1].set_title('Month_Count')
plt.suptitle('월별 구매자', fontsize=20)
plt.show()

- 전체 구매자의 월별 구매비율

In [ ]:
f,ax = plt.subplots(1,2,figsize=(16,8))
df1['date'].value_counts().plot.pie(autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Month Percentage')
ax[0].set_ylabel('')
sns.countplot('date', data=df1, ax=ax[1])
ax[1].set_title('Month_Count')
plt.suptitle('재구매 이력이 있는 구매자', fontsize=20)
plt.show()

In [ ]:
f,ax = plt.subplots(1,2,figsize=(16,8))
df4['date'].value_counts().plot.pie(autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Month Percentage')
ax[0].set_ylabel('')
sns.countplot('date', data=df4, ax=ax[1])
ax[1].set_title('Month_Count')
plt.show()

- [Welcome]쿠폰을 사용한적이 있는 재구매 이력이 있는 구매자의 월별 구매량

In [ ]:
f,ax = plt.subplots(1,2,figsize=(20,10))
df4['category_title'].value_counts().plot.pie(autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Month Percentage')
ax[0].set_ylabel('')
sns.countplot('category_title', data=df4, ax=ax[1])
ax[1].set_title('Month_Count')
plt.show()

쿠폰을 사용한적이 있는 재구매 이력이 있는 구매자의 카테고리

In [ ]:
df[df.sale_price.isnull()]

In [ ]:
nantitle = df.drop(df[df.course_title.isna()].index)